
# Getting Started - Vertex AI GenAI Python Client

How to get information into (prompts) and out of (responses) of Vertex AI hosted Generative AI Models!

Vertex AI has access to hosted [generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) models, called foundation models. These can be easily interacted with in the console using [Vertex AI Generative AI Studio](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio). This notebook covers interacting with the models with the [`vertexai`](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) Python package that is include with the Vertex AI SDK [aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest).

*   Gemini API (Multimodal data, text, code, chat)
    *   `vertexai.generative_models`
*   PaLM API (Text, chat, embeddings)
    *   `vertexai.language_models`
*   Codey APIs (Code generation, code chat, code completion)
    *   `vertexai.language_models`
*   Imagen API (Image generation, image editing, image captioning, visual question answering, and multimodal embedding)
    *   `vertexai.vision_models`

The current models in GA and in preview can be reviewed in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models) as well as within the [Vertex AI Model Garden](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-garden) which also include access to serving and tuning OSS models.

Model have names and versions. The [names](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#model_naming_scheme) are designed show both the purpose and size of the model. For instance, sizes are conveyed with names like `Bison` and `Gecko` while purposes are `text`, `textembedding`, `chat`, `code`, `imagetext`, `imagegeneration` and more. [Versions](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-versioning) are either `stable` and available for six months after the release of subsequent version, or `latest` which are incremenatally updated and improved but can give different responses to the same prompt.

## Generative AI



## Refs

* [Use text models and the Vertex AI SDK  |  Generative AI on Vertex AI  |  Google Cloud](https://cloud.google.com/vertex-ai/generative-ai/docs/sdk-for-llm/sdk-use-text-models)
* [Introduction to language model classes in the Vertex AI SDK  |  Generative AI on Vertex AI  |  Google Cloud](https://cloud.google.com/vertex-ai/generative-ai/docs/sdk-for-llm/llm-sdk-overview)


* [Discovery Engine v1beta API - Class SearchRequest (1.0.0-beta09)  |  .NET client library  |  Google Cloud](https://cloud.google.com/dotnet/docs/reference/Google.Cloud.DiscoveryEngine.V1Beta/latest/Google.Cloud.DiscoveryEngine.V1Beta.SearchRequest)
* [Package google.cloud.discoveryengine.v1alpha  |  Vertex AI Search and Conversation  |  Google Cloud](https://cloud.google.com/generative-ai-app-builder/docs/reference/rpc/google.cloud.discoveryengine.v1alpha#searchrequest)

* [SearchResponse  |  Vertex AI Search and Conversation  |  Google Cloud](https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1alpha/SearchResponse)



* [Model versions and lifecycle  |  Generative AI on Vertex AI  |  Google Cloud](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versioning)

## other 

https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Getting%20Started%20-%20Vertex%20AI%20GenAI%20Python%20Client.ipynb



In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
print(PROJECT_ID)

## installs


In [8]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [3]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'demos-vertex-ai'

In [4]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'getting-started'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

In [15]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import requests
import IPython

In [16]:
aiplatform.__version__

'1.38.1'

In [17]:

vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

In [18]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

AttributeError: module 'vertexai.generative_models' has no attribute 'GenerativeModel'

In [ ]:
prompt = "What are the starting points for AI?"

In [14]:
response = gemini_text.generate_content(prompt)
response

NameError: name 'gemini_text' is not defined